In [50]:
from rekall import Interval, IntervalSet, IntervalSetMapping, Bounds3D
from rekall.predicates import *
from rekall.stdlib import ingest
from vgrid import VGridSpec, VideoMetadata, VideoBlockFormat, FlatFormat
from vgrid_jupyter import VGridWidget
import urllib3, requests, os

In [2]:
urllib3.disable_warnings()

# Video metadata

In [26]:
VIDEO_COLLECTION_BASEURL = "http://olimar.stanford.edu/hdd/tvnews-sandbox"
VIDEO_ENDPOINT = "http://olimar.stanford.edu/hdd/tvnews-sandbox/videos"
VIDEO_METADATA_FILENAME = "data/video_meta_sandbox.json"

In [27]:
req = requests.get(os.path.join(VIDEO_COLLECTION_BASEURL, VIDEO_METADATA_FILENAME), verify=False)
video_collection = req.json()

In [28]:
video_metadata = [
    VideoMetadata(v["path"], v["id"], v["fps"], int(v["num_frames"]), v["width"], v["height"])
    for v in video_collection
]

In [29]:
vgrid_spec = VGridSpec(
    video_meta = video_metadata,
    vis_format = VideoBlockFormat(imaps = None, video_meta = video_metadata),
    video_endpoint = os.path.join(VIDEO_COLLECTION_BASEURL, 'videos')
)
VGridWidget(vgrid_spec = vgrid_spec.to_json_compressed())

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xe5]mo\xdb8\x12\xfe+\x8b|\xbeK\xf8*\x8a\xfd\xb6I\x…

# Load faces

In [23]:
video_meta_by_id = {
    vm.id: vm
    for vm in video_metadata
}

In [12]:
FACES_JSON = "data/face_dump.json"
req = requests.get(os.path.join(VIDEO_COLLECTION_BASEURL, FACES_JSON), verify=False)
faces_json = req.json()

In [19]:
faces_ism = ingest.ism_from_iterable_with_schema_bounds3D(
    faces_json,
    ingest.getter_accessor,
    {
        'key': 'video_id',
        't1': 'frame_number',
        't2': 'frame_number',
        'x1': 'x1',
        'x2': 'x2',
        'y1': 'y1',
        'y2': 'y2'
    },
    with_payload = lambda item: {
        'face': item,
        'video': video_meta_by_id[item['video_id']]
    },
    progress = True
)

100%|██████████| 218006/218006 [00:02<00:00, 95021.16it/s] 


In [21]:
faces_ism = faces_ism.map(
    lambda face: Interval(
        Bounds3D(
            face['t1'] / face['payload']['video'].fps - 1.5,
            face['t2'] / face['payload']['video'].fps + 1.5,
            face['x1'],
            face['x2'],
            face['y1'],
            face['y2']
        ),
        face['payload']['face']
    )
)

In [22]:
vgrid_spec = VGridSpec(
    video_meta = video_metadata,
    vis_format = VideoBlockFormat(imaps = [
        ('faces', faces_ism)
    ]),
    video_endpoint = os.path.join(VIDEO_COLLECTION_BASEURL, 'videos')
)
VGridWidget(vgrid_spec = vgrid_spec.to_json_compressed())

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcc\xbd\xdd\xaedIr\x9d\xf9*B_\x8f\x0e\xfc\xffG\x97…

# Load Captions

In [38]:
CAPTIONS_JSON = "data/captions.json"

In [39]:
def load_json(video_baseurl, json_path):
    req = requests.get(os.path.join(video_baseurl, json_path), verify=False)
    json_objs = req.json()
    ism = ingest.ism_from_iterable_with_schema_bounds3D(
        json_objs,
        ingest.getter_accessor,
        {
            'key': 'video_id',
            't1': 'start',
            't2': 'end'
        },
        with_payload = lambda item: item,
        progress = True
    )
    return ism

In [57]:
captions = load_json(VIDEO_COLLECTION_BASEURL, CAPTIONS_JSON).map(
    lambda caption: Interval(caption['bounds'], caption['payload']['caption'])
).coalesce(
    ('t1', 't2'),
    Bounds3D.span,
    lambda p1, p2: p1 + ' ' + p2,
    predicate = lambda i1, i2: '>>' not in i2['payload'],
    epsilon = 1.0
)

100%|██████████| 836546/836546 [00:08<00:00, 94580.64it/s] 


In [42]:
def vgrid_captions(caption_ism):
    from vgrid import SpatialType_Caption
    
    return caption_ism.map(
        lambda caption: Interval(
            caption['bounds'],
            {
                'spatial_type': SpatialType_Caption(caption['payload']),
                'metadata': {}
            }
        )
    )

In [61]:
vgrid_spec = VGridSpec(
    video_meta = video_metadata,
    vis_format = VideoBlockFormat(imaps = [
        ('faces', faces_ism),
        ('_captions', vgrid_captions(captions))
    ]),
    video_endpoint = os.path.join(VIDEO_COLLECTION_BASEURL, 'videos')
)
VGridWidget(vgrid_spec = vgrid_spec.to_json_compressed())

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcc\xbd\xdd\xaedIr\x9d\xf9*B_\x8f\x0e\xfc\xffG\x97…

# Load Interview, Commercial, Panel Annotations

In [30]:
INTERVIEW_JSON = "data/interviews.json"
COMMERCIAL_JSON = "data/commercials.json"
PANELS_JSON = "data/panels.json"

In [36]:
interviews = load_json(VIDEO_COLLECTION_BASEURL, INTERVIEW_JSON)
commercials = load_json(VIDEO_COLLECTION_BASEURL, COMMERCIAL_JSON)
panels = load_json(VIDEO_COLLECTION_BASEURL, PANELS_JSON)

100%|██████████| 135/135 [00:00<00:00, 236422.15it/s]


In [63]:
vgrid_spec = VGridSpec(
    video_meta = video_metadata,
    vis_format = VideoBlockFormat(imaps = [
        ('faces', faces_ism),
        ('interviews', interviews),
        ('commercials', commercials),
        ('panels', panels),
        ('_captions', vgrid_captions(captions))
    ]),
    video_endpoint = os.path.join(VIDEO_COLLECTION_BASEURL, 'videos')
)
VGridWidget(vgrid_spec = vgrid_spec.to_json_compressed())

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcc\xbd\xdd\xaedIr\x9d\xf9*B_\x8f\x0e\xfc\xffG\x97…

# Load Face Features

Example of how to load face features.

In [68]:
import pickle

In [64]:
video_id = video_metadata[0].id

In [73]:
FEATURE_PATH = 'data/face_features/{}.pkl'.format(video_id)
req = requests.get(os.path.join(VIDEO_COLLECTION_BASEURL, FEATURE_PATH), verify=False)

In [75]:
features = pickle.loads(req.content)

In [78]:
faces_with_features = faces_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        intrvl['payload'].update({
            'features': features[intrvl['payload']['id']]
        } if intrvl['payload']['id'] in features else {})
    ) 
)